In [319]:
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math, random, torch, collections, time, torch.nn.functional as F, networkx as nx, matplotlib.pyplot as plt, numpy as np
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from IPython.display import clear_output
from torch_geometric.utils import to_networkx
from torch_geometric.utils import from_networkx

In [320]:
import sys, os
sys.path.append('../../gnumap/')
from models.train_models import *
from scipy import optimize

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from codecarbon import OfflineEmissionsTracker
from umap_functions import *
from simulation_utils import make_roll

In [ ]:
N_NEIGHBOURS = 5


In [ ]:
[x,y,z], t, new_data = make_roll(n_neighbours = N_NEIGHBOURS, scale=0.1, n_samples = 4000, features='coordinates')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(x, y, z, c=t, cmap='Spectral')

In [ ]:
# import umap.umap_ as umap
import umap
import numpy as np
from sklearn.neighbors import kneighbors_graph

X = np.vstack([np.array(x),np.array(y),np.array(z)]).T
A_dist = kneighbors_graph(X, N_NEIGHBOURS, mode='distance', include_self=False)
embedding = umap.UMAP(n_components=2, n_neighbors= 10, min_dist= 0.3).fit_transform(X)
plt.scatter(*embedding.T, s=10, c=t, alpha=0.5, cmap='Spectral')

In [ ]:
# more local when n_neighbors is small
embedding_20 = umap.UMAP(n_components=2, n_neighbors= 20, min_dist= 0.1).fit_transform(X)
plt.scatter(*embedding_20.T, s=10, c=t, alpha=0.5, cmap='Spectral')

In [ ]:
# more global when n_neighbors is large
embedding_200 = umap.UMAP(n_components=2, n_neighbors= 200, min_dist= 0.1).fit_transform(X)
plt.scatter(*embedding_200.T, s=10, c=t, alpha=0.5, cmap='Spectral')

In [ ]:
A_dist[:,0]

In [ ]:
new_data.x # coordinate to node features

In [ ]:
X

In [ ]:
# A_dist = kneighbors_graph(X, 5, mode='distance', include_self=False)
### Very sensitive to wrong edges
plt.figure()
nx.draw_networkx(nx.from_scipy_sparse_matrix(A_dist), 
                 pos={i:[new_data.x[i,0].numpy(),new_data.x[i,1].numpy()] for i in range(new_data.num_nodes)},
                 # to see the "roll" it should by x, y axis
                 node_color=t, cmap='Spectral', with_labels=False)
plt.show()

In [ ]:
model = train_cca_ssg(new_data, channels=2, hid_dim=256, lambd=1e2,
                  n_layers=2, epochs=1000, lr=1e-2,
                  fmr=0.2, edr =0.4, name_file="test",
                  device=None)
plt.hist(model.get_embedding(new_data).numpy())
plt.show()
plt.figure()
out = model.get_embedding(new_data).numpy()
u = out
plt.scatter(u[:,0], u[:,1], c = t, 
            cmap="Spectral")
plt.show()

In [ ]:
# Deep Graph Infomax
https://github.com/PetarV-/DGI

In [ ]:
# First, create "results" folder at current working directory
model = train_dgi(new_data,512, 2, 2, patience=20,
              epochs=200, lr=1e-3, name_file="1")
plt.hist(model.get_embedding(new_data).numpy())
plt.figure()
out = model.get_embedding(new_data).numpy()
u = out
plt.scatter(u[:,0], u[:,1], c = t, 
            cmap="Spectral")
plt.show()

In [ ]:
from numbers import Number
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from models.aggregation import GAPPNP
class GNN(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, n_layers: int,
                 activation: str='relu', slope: float=.1,
                 device: str='cpu',
                 alpha_res: float=0, alpha: float=0.5,
                 beta: float=1., gnn_type: str = 'symmetric',
                 norm: str='normalize',
                 must_propagate=None,
                 lambd_corr: float = 0):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.gnn_type = gnn_type
        self.n_layers = n_layers
        self.device = device
        self.alpha_res = alpha_res
        self.alpha = alpha
        self.beta= beta
        self.must_propagate = must_propagate
        self.propagate = GAPPNP(K=1, alpha_res=self.alpha_res,
                                alpha = self.alpha,
                                gnn_type=self.gnn_type,
                                beta = self.beta)
        self.norm = norm
        if self.must_propagate is None:
            self.must_propagate = [True] * self.n_layers
        if isinstance(hidden_dim, Number):
            self.hidden_dim = [hidden_dim] * (self.n_layers - 1)
        elif isinstance(hidden_dim, list):
            self.hidden_dim = hidden_dim
        else:
            raise ValueError('Wrong argument type for hidden_dim: {}'.format(hidden_dim))

        if isinstance(activation, str):
            self.activation = [activation] * (self.n_layers - 1)
        elif isinstance(activation, list):
            self.hidden_dim = activation
        else:
            raise ValueError('Wrong argument type for activation: {}'.format(activation))

        self._act_f = []
        for act in self.activation:
            if act == 'lrelu':
                self._act_f.append(lambda x: F.leaky_relu(x, negative_slope=slope))
            elif act == 'relu':
                self._act_f.append(lambda x: torch.nn.ReLU()(x))
            elif act == 'xtanh':
                self._act_f.append(lambda x: self.xtanh(x, alpha=slope))
            elif act == 'sigmoid':
                self._act_f.append(F.sigmoid)
            elif act == 'none':
                self._act_f.append(lambda x: x)
            else:
                ValueError('Incorrect activation: {}'.format(act))

        if self.n_layers == 1:
            _fc_list = [nn.Linear(self.input_dim, self.output_dim)]
        else:
            _fc_list = [nn.Linear(self.input_dim, self.hidden_dim[0])]
            for i in range(1, self.n_layers - 1):
                _fc_list.append(nn.Linear(self.hidden_dim[i - 1], self.hidden_dim[i]))
            _fc_list.append(nn.Linear(self.hidden_dim[self.n_layers - 2], self.output_dim))
        self.fc = nn.ModuleList(_fc_list)
        self.to(self.device)

    @staticmethod
    def xtanh(x, alpha=.1):
        """tanh function plus an additional linear term"""
        return x.tanh() + alpha * x

    def forward(self, x, edge_index):
        h = x
        for c in range(self.n_layers):
            if c == self.n_layers - 1:
                h = self.fc[c](h)
                if self.norm == 'normalize' and c==0:
                    h = F.normalize(h, p=2, dim=1)
                elif self.norm == 'standardize'and c==0:
                    h = (h - h.mean(0)) / h.std(0)
                elif self.norm == 'uniform'and c==0:
                    h = 1 * (h - h.min()) / (h.max() - h.min())
                elif self.norm == 'col_uniform'and c==0:
                    h = 1 * (h - h.min(0)[0].reshape([1,-1]))/ (h.max(0)[0].reshape([1,-1])-h.min(0)[0].reshape([1,-1]))

            else:
                h = self.fc[c](h)
                h = F.dropout(h, p=0.5, training=self.training)
                if self.must_propagate[c]:
                    h = self.propagate(h, edge_index)
                if self.norm == 'normalize':
                    h = F.normalize(h, p=2, dim=1)
                elif self.norm == 'standardize':
                    h = (h - h.mean(0)) / h.std(0) #z1 = (h1 - h1.mean(0)) / h1.std(0)
                elif self.norm == 'uniform':
                    h = 1 * (h - h.min()) / (h.max() - h.min())
                elif self.norm == 'col_uniform':
                    h = 1 * (h - h.min(0)[0].reshape([1,-1]))/ (h.max(0)[0].reshape([1,-1])-h.min(0)[0].reshape([1,-1]))
                h = self._act_f[c](h)
        if self.norm == 'standardize_last':
            h = (h - h.mean(0)) / h.std(0)
        return h


# GNUMAP

densmap reference: https://www.biorxiv.org/content/10.1101/2020.05.12.077776v1.full.pdf
$ L^{denseMAP} = CE(P||Q) - \lambda Corr(r_o^{UMAP}, r_e^{UMAP}) $

In [ ]:
import numpy as np
from carbontracker.tracker import CarbonTracker
import cProfile
import os
import scipy
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import remove_self_loops, negative_sampling
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import to_scipy_sparse_matrix, to_networkx, from_scipy_sparse_matrix
import time
from umap_functions import *

def train_gnumap(data, hid_dim, dim, n_layers=2, target=None,
                 method = 'laplacian', must_propagate=None,
                 norm='normalize', neighbours=15,
                 patience=20, epochs=200, lr=1e-3, wd=1e-2,
                 min_dist=0.1, name_file="1", subsampling=None,
                 alpha: float=0.5, spread = 1.0, lambd_corr=1e-2,
                 beta: float=1., gnn_type: str = 'symmetric',
                 repulsion_strength=None,
                 local_connectivity=1,
                 device=None, colours=None):


    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    EPS_0 = data.num_edges/ (data.num_nodes ** 2)
    _a, _b = find_ab_params(spread, min_dist) # spread , min_dist given as hyperparameter

    #if torch_geometric.utils.is_undirected(data.edge_index):
    #    new_edge_index, new_edge_attr = torch_geometric.utils.to_undirected(data.edge_index, data.edge_weight)
    #else:
    #    new_edge_index, new_edge_attr = data.edge_index, data.edge_weight

    if torch_geometric.utils.is_undirected(data.edge_index):
        new_edge_index, new_edge_attr = data.edge_index, data.edge_weight
    else:
        new_edge_index, new_edge_attr = torch_geometric.utils.to_undirected(data.edge_index, data.edge_weight)

    #### transform edge index into knn matrix
    knn = []
    for i in range(data.num_nodes):
        knn += [list(np.sort(list(new_edge_attr[(new_edge_index[0]==i) & (new_edge_index[1]!=i )].numpy())))]
    knn_dists = pd.DataFrame(knn).fillna(0).values
    sigmas, rhos = smooth_knn_dist(
            knn_dists,
            float(neighbours),
            local_connectivity=float(local_connectivity),
             )

    # Maybe use the distance as the original distribution?
    vals = [ np.exp(-(np.max([new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0])) /
                    (sigmas[new_edge_index[0,i]])) for i in range(len(new_edge_attr))]

    #print(np.where(vals > 1e5))
    rows = new_edge_index[0,:].numpy()
    cols = new_edge_index[1,:].numpy()
    vals = np.array(vals)
    vals[vals<1e-5] = 0

    high = []
    for i in range(data.num_nodes):
        high.append(
            np.insert((new_edge_attr[(new_edge_index[0]==i) & (new_edge_index[1]!=i )].numpy())/new_edge_attr[(new_edge_index[0] == i) & (new_edge_index[1] != i)].sum().numpy(),0,0)
        )
    # highs = np.hstack(high)
    highs  = new_edge_attr/data.num_edges
    p =[]
    for i in range(data.num_nodes):
        p.append(
            highs[(new_edge_index[0] == i) & (new_edge_index[1] != i)].sum().numpy()
        )
    eta = data.edge_weight
    for i in range(len(data.edge_weight)):
        eta[i] = (p[data.edge_index[0,i]]+p[data.edge_index[1,i]])/2*data.x.shape[0]

    result = scipy.sparse.coo_matrix(
  #      (vals, (rows, cols)), shape=(X.shape[0], X.shape[0])
         (highs, (rows, cols)), shape=(X.shape[0], X.shape[0])
    )
    result.eliminate_zeros()
    target_graph_index, target_graph_weights = from_scipy_sparse_matrix(result)



    #### Prune
    EPS = 1e-29 #math.exp(-1.0/(2*_b) * math.log(1.0/_a * (1.0/EPS_0 -1)))
    print("Epsilon is " + str(EPS))
    print("Hyperparameters a = " + str(_a) + " and b = " + str(_b))



    model = GNN(data.num_features, hid_dim, dim, n_layers=n_layers,
                must_propagate=must_propagate,
                norm=norm)
    model = model.to(device)
    model.apply(init_weights)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                             weight_decay=wd)
    new_data = Data(x=data.x, edge_index=target_graph_index,
                    y=data.y, edge_weight=target_graph_weights)
    sparsity =  new_data.num_edges/(new_data.num_nodes**2 -new_data.num_nodes)
    if repulsion_strength is None:
        repulsion_strength = 1.0/sparsity
        # we have way more samples that are "not" connected(sparsity), so need to give more weight to negative sampling to get balanced results
    row_pos, col_pos =  new_data.edge_index
    index = (row_pos != col_pos)
        # to exclude self-connectivity
    edge_weights_pos = new_data.edge_weight#[index]

    if target is not None:
        edge_weights_pos = fast_intersection(row_pos[index], col_pos[index], edge_weights_pos,
                                             target, unknown_dist=1.0, far_dist=5.0)
        # p_{ij}
    if subsampling is None:
        row_neg, col_neg = negative_sampling(new_data.edge_index, num_neg_samples = 5 * new_data.edge_index.shape[1] )
        # m = 5
        index_neg = (row_neg != col_neg)
        # edge_weights_neg = EPS * torch.ones(len(row_neg))
        edge_weights_neg = m*torch.ones(len(row_neg))
        if target is not None:
            edge_weights_neg = fast_intersection(row_neg[index_neg], col_neg[index_neg], edge_weights_neg,
                                                 target, unknown_dist=1.0, far_dist=5.0)
    best_t=0
    cnt_wait = 0
    best=1e9
    log_sigmoid = torch.nn.LogSigmoid()
    edges = [(e[0],e[1]) for _, e in enumerate(data.edge_index.numpy().T)]
    for epoch in range(epochs):
        tic_epoch = time.time()
        model.train()
        optimizer.zero_grad()
        tic = time.time()
        out = model(data.x.float(), data.edge_index)
        diff_norm = torch.sum(torch.square(out[row_pos[index]] - out[col_pos[index]]), 1)
        diff_norm = torch.clip(diff_norm, min=1e-3)
        log_q = - torch.log1p(_a *  diff_norm ** _b) # 1/(1+a*d^2b)
        # log_q = - torch.log1p(1+ diff_norm)
        loss_pos = - torch.mean(edge_weights_pos[index] * log_sigmoid(log_q)) - torch.mean((1. - edge_weights_pos[index]) *  (log_sigmoid(log_q) - log_q ) * repulsion_strength)
        # log(q/(q+1))


        if subsampling is None:
            diff_norm_neg = torch.sum(torch.square(out[row_neg[index_neg]] - out[col_neg[index_neg]]), 1) #+ 1e-3
            diff_norm_neg = torch.clip(diff_norm_neg, min=1e-3)
            log_q_neg = - torch.log1p(_a *  diff_norm_neg ** _b)
            # log_q_neg = - torch.log1p(1+ diff_norm_neg)
        else:
            row_neg, col_neg = negative_sampling(new_data.edge_index,
                                                 num_neg_samples=subsampling)
            index_neg = (row_neg != col_neg)
            edge_weights_neg = EPS * torch.ones(len(row_neg))
            if target is not None:
                edge_weights_neg = fast_intersection(row_neg[index_neg],
                                                     col_neg[index_neg], edge_weights_neg,
                                                     target, unknown_dist=1.0, far_dist=5.0)
            diff_norm_neg = torch.sum(torch.square(out[row_neg[index_neg]] - out[col_neg[index_neg]]), 1) #+ 1e-3
            diff_norm_neg = torch.clip(diff_norm_neg, min=1e-3)
            log_q_neg = - torch.log1p(_a *  diff_norm_neg ** _b)
            # log_q_neg = - torch.log1p(1+ diff_norm_neg)
        print("loss before neg", loss_pos)
        loss_neg = - torch.mean((log_sigmoid(log_q_neg) - log_q_neg ) * repulsion_strength)
        print("loss after neg", loss_neg)
        ### Add a term to make sure that the features are learned independently
        c1 = torch.mm(out.T, out)
        c1 = c1 / out.shape[0]
        iden = torch.tensor(np.eye(out.shape[1])).to(device)
        loss_dec1 = (torch.diag_embed(c1) - c1).pow(2).sum()
        loss = loss_pos + loss_neg +  lambd_corr * loss_dec1
        print("loss corr", lambd_corr * loss_dec1)
        print("loss final", loss)
        tic =  time.time()
        loss.backward()
        #torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=4)
        optimizer.step()
        
        if epoch%10== 0:
            u = out.detach().numpy()
            plt.figure()
            plt.scatter(u[:,0], u[:,1], c = t, 
                        cmap="Spectral")
            plt.show()
            print(torch.mm(out.T, out)/ new_data.num_nodes)

        for g in optimizer.param_groups:
            g['lr'] = lr * (1.0 - (float(epoch) / float(epochs)))

        print('Epoch={:03d}, loss={:.4f}, time={:.4f}'.format(epoch, loss.item(),time.time()-tic_epoch))
        if loss < best:
            best = loss
            best_t = epoch
            cnt_wait = 0
            torch.save(model.state_dict(), os.getcwd()  + '/results/best_gnumap_'
                                          + str(method) + '_neigh' + str(neighbours)
                                          + '_dim' + str(dim) + '_' + name_file +  '.pkl')
        else:
            cnt_wait += 1
        if cnt_wait == patience and epoch>50:
            print('Early stopping at epoch {}!'.format(epoch))
            break
        #print("Time epoch after saving", time.time()-tic_epoch)
    #tracker.stop()
    print('Loading {}th epoch'.format(best_t))
    model.load_state_dict(torch.load(os.getcwd()  + '/results/best_gnumap_' +
                                     str(method) + '_neigh' + str(neighbours)
                                     + '_dim' + str(dim) + '_' + name_file + '.pkl'))
    return(model,target_graph_index, vals, knn_dists)

In [ ]:
model2, target_graph_index, vals, knn_dists=  train_gnumap(new_data, 
                                     target=None, hid_dim=1054, dim=2, 
                                     n_layers=3, must_propagate= [True, True, True, True, True],
                                     method = 'laplacian', alpha=0.5, 
                                     gnn_type='symmetric', repulsion_strength=100,
                                     norm='uniform', neighbours=5,
                                      beta=1, patience=20, epochs=1000,
                                      lr=1e-1, 
                                    wd=1e-4,
                                    lambd_corr=1e-4,
                                       min_dist=0.01,
                       subsampling=100000)
out = model2(new_data.x.float(), new_data.edge_index)
plt.figure()
plt.hist(out.detach().numpy())
plt.show()
u = out.detach().numpy()
plt.figure()
plt.scatter(u[:,0], u[:,1], c = t, 
            cmap="Spectral")
plt.show()
print(torch.mm(out.T, out)/ new_data.num_nodes)

In [ ]:
plt.hist(vals)

In [ ]:
new_edge_index, new_edge_attr = remove_self_loops(new_data.edge_index, new_data.edge_weight)

In [ ]:
knn_dists
sigmas, rhos = smooth_knn_dist(
            knn_dists,
            float(2),
            local_connectivity=float(1.),
             )

In [ ]:
vals = []
for i in range(len(new_edge_attr))[:100]:
    vals+= [ np.exp(-(np.max([new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0])) /
                    (sigmas[new_edge_index[0,i]]))]
    print([i, np.exp(-(np.max([new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0])) /
                    (sigmas[new_edge_index[0,i]]))])

In [ ]:
i = 5
print(new_edge_index[:,i], new_edge_attr.numpy()[i] , rhos[new_edge_index[0,i]], sigmas[new_edge_index[0,i]])
print(np.max(new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0)/sigmas[new_edge_index[0,i]])

In [ ]:
np.max([new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0]
      )

In [ ]:
np.max(new_edge_attr.numpy()[i] - rhos[new_edge_index[0,i]], 0)/sigmas[new_edge_index[0,i]]

In [ ]:
plt.hist(rhos)

In [ ]:
#xx = torch.ones((X.shape[0], 10))
new_data3 = copy.deepcopy(new_data)
# new_data2.x =  new_data.x[:,:2] # leave out z-axis
model2, target_index =  train_gnumap(new_data3,
                                     target=None, hid_dim=256, dim=2, 
                                     n_layers=1, must_propagate= [True, True, True, True, True],
                                     method = 'laplacian', alpha=0.5, 
                                     gnn_type='symmetric', repulsion_strength=1.,
                                     norm='standardize', neighbours=2,
                                    beta=1, patience=20, epochs=1000,
                                       lr=1e-2, 
                                       wd=1e-4,
                                        lambd_corr=1.,
                                       min_dist=0.001,
                       subsampling=100000)
out = model2(new_data3.x.float(), new_data.edge_index)
plt.figure()
plt.hist(out.detach().numpy())
plt.show()
u = out.detach().numpy()
plt.figure()
plt.scatter(u[:,0], u[:,1], c = t, 
            cmap="Spectral")
plt.show()
print(torch.mm(out.T, out)/ new_data.num_nodes)